In [1]:
import os
from dotenv import load_dotenv
import time
from datetime import datetime
import schedule
import requests

import pandas as pd
import pandas_ta as ta
import numpy as np

from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.foreignexchange import ForeignExchange
load_dotenv()

True

In [2]:
API_KEYS = os.getenv('ALPHA_KEYS')
cc = ForeignExchange(key=API_KEYS)

LINE_TOKEN = os.getenv('LINE_TOKEN')
LINE_URL = 'https://notify-api.line.me/api/notify'
LINE_HEADERS = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer ' + LINE_TOKEN }

In [3]:
data, _ = cc.get_currency_exchange_rate(from_currency='EUR',to_currency='USD')
ts = TimeSeries(key=API_KEYS, output_format='pandas')

In [4]:
data

{'1. From_Currency Code': 'EUR',
 '2. From_Currency Name': 'Euro',
 '3. To_Currency Code': 'USD',
 '4. To_Currency Name': 'United States Dollar',
 '5. Exchange Rate': '1.13379000',
 '6. Last Refreshed': '2022-01-03 05:44:01',
 '7. Time Zone': 'UTC',
 '8. Bid Price': '1.13379000',
 '9. Ask Price': '1.13379000'}

In [5]:
data_ts,_ = ts.get_intraday(symbol='EURUSD',interval='30min', outputsize='compact')
data_ts

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2022-01-03 01:00:00,1.1337,1.1338,1.1334,1.1337,0.0
2022-01-03 00:30:00,1.1340,1.1341,1.1334,1.1337,0.0
2022-01-03 00:00:00,1.1348,1.1348,1.1337,1.1340,0.0
2022-01-02 23:30:00,1.1348,1.1349,1.1341,1.1348,0.0
2022-01-02 23:00:00,1.1344,1.1350,1.1339,1.1348,0.0
...,...,...,...,...,...
2021-12-30 01:00:00,1.1331,1.1331,1.1323,1.1325,0.0
2021-12-30 00:30:00,1.1333,1.1333,1.1328,1.1331,0.0
2021-12-30 00:00:00,1.1335,1.1335,1.1330,1.1333,0.0


In [6]:
### line alert function
def notifyMsg(*msg, url=LINE_URL, headers=LINE_HEADERS):
    r = requests.post(url, headers=headers, data={'message': msg[0]})
    print(r.text)

In [7]:
### create technical strategy
rsi_o2 = ta.Strategy(
    name = 'RSI 14 Days for opposite trade',
    description = 'RSI for alert',
    ta = [
        {'kind': 'ema', 'length': 200},
        {'kind': 'rsi', 'length': 14}
    ]
)

In [8]:
def get_realtime(pair='EURUSD', tf='15min'):
    bars, _ = ts.get_intraday(symbol='EURUSD',interval='15min',  outputsize='full')
    df = pd.DataFrame({
        'timestamp': bars.index,
        'open': bars['1. open'],
        'high': bars['2. high'],
        'low': bars['3. low'],
        'close': bars['4. close'],
        'volumn': bars['5. volume']
    }) # convert alpha vantage data to use with rsi strategy
    
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df = df.iloc[::-1] # reverse row
    df.reset_index(drop=True, inplace=True)
    df.ta.strategy(rsi_o2)
    
    # loop count latest n bars that rsi < 30 or rsi > 70
    sell_point = 0
    buy_point = 0
    n_bars = 1  # number of latest bars to backword lookup
    for index, row in df.tail(n_bars).iterrows():
        if (row['RSI_14'] < 30 and row['EMA_200'] > row['high']): # rsi < 30 and ema 200 > high price then send sell signal
            sell_point = sell_point + 1
        elif (row['RSI_14'] > 70 and row['EMA_200'] < row['low']): # if rsi > 70 and ema 200 < low price then send buy signal
            buy_point = buy_point + 1
    
        if buy_point >= n_bars:
            notifyMsg('Buy EURUSD price: ' + str(row['open']))
        elif sell_point >= n_bars:
            notifyMsg('Sell EURUSD price: ' + str(row['open']))
    
# get_realtime()

In [9]:
schedule.every(60 * 15).seconds.do(get_realtime)

Every 900 seconds do get_realtime() (last run: [never], next run: 2022-01-03 12:59:16)

In [10]:
while True:
    schedule.run_pending()
    time.sleep(1)

{"status":200,"message":"ok"}
{"status":200,"message":"ok"}
{"status":200,"message":"ok"}
{"status":200,"message":"ok"}
{"status":200,"message":"ok"}
{"status":200,"message":"ok"}
{"status":200,"message":"ok"}


KeyboardInterrupt: 